# **Chapter 6: Asynchronous Programming**

---

## **Learning Objectives**

By the end of this chapter, you will be able to:

- Understand Dart's event loop model and how asynchronous code executes
- Create and use Futures for single-completion asynchronous operations
- Implement async/await patterns for readable asynchronous code
- Handle errors in asynchronous operations properly
- Work with Streams for multiple-event asynchronous operations
- Use StreamBuilder to build responsive UIs in Flutter
- Transform Streams using operators like map, where, and expand
- Understand when and how to use Isolates for CPU-intensive tasks
- Write efficient, non-blocking Flutter applications

---

## **Prerequisites**

- Completed Chapter 5: Object-Oriented Dart
- Understanding of functions, callbacks, and event-driven programming
- Basic knowledge of asynchronous concepts (callbacks, promises)

---

## **6.1 Understanding Asynchronous Programming**

Asynchronous programming allows your application to perform long-running operations (like network requests, file I/O, or database queries) without blocking the UI thread. This is crucial for Flutter applications, as blocking the main thread causes the UI to freeze, leading to a poor user experience.

### **The Dart Event Loop**

Dart uses a single-threaded execution model with an event loop. Understanding the event loop is essential for writing efficient asynchronous code.

```dart
void main() {
  print('1. Synchronous operation started');
  
  // Synchronous code executes immediately
  performSyncTask();
  
  print('2. Scheduling async operations');
  
  // Schedule asynchronous operations
  scheduleMicrotask(() {
    print('3. Microtask 1 executes');
  });
  
  Future(() {
    print('4. Future 1 executes');
  });
  
  Future(() {
    print('5. Future 2 executes');
  });
  
  scheduleMicrotask(() {
    print('6. Microtask 2 executes');
  });
  
  print('7. Synchronous code continues');
  
  // Another synchronous operation
  printData();
}

void performSyncTask() {
  print('   Performing synchronous task');
  // This code runs immediately and blocks execution
  // until it completes
}

void printData() {
  print('   Printing data synchronously');
}

/*
Output Order:
1. Synchronous operation started
   Performing synchronous task
2. Scheduling async operations
7. Synchronous code continues
   Printing data synchronously
3. Microtask 1 executes
6. Microtask 2 executes
4. Future 1 executes
5. Future 2 executes
*/
```

**Explanation:**

- **Event loop execution order**: Dart executes code in this order:
  1. All synchronous code in the `main()` function executes first
  2. Microtasks are then executed (scheduled via `scheduleMicrotask`)
  3. Event loop tasks (Futures) are executed last
- **`scheduleMicrotask()`**: Queues a microtask to execute after the current synchronous code completes but before the next event loop iteration. Microtasks have higher priority than regular Futures.
- **`Future()`**: Queues a task to execute in the next event loop iteration after all microtasks have completed.
- **Microtask queue vs. Event queue**: The microtask queue is processed before the event queue. If you add microtasks within microtasks, they all execute before any event queue tasks.
- **Single-threaded**: Dart is single-threaded, meaning only one piece of code executes at a time. Asynchronous operations don't run concurrently; they're scheduled to run later.
- **Why this matters**: Understanding this execution order helps you predict when your code will run and avoid race conditions or unexpected behavior.

### **Synchronous vs. Asynchronous Operations**

```dart
import 'dart:io';

void main() {
  print('=== Synchronous Operation ===');
  syncOperation();
  
  print('\n=== Asynchronous Operation ===');
  asyncOperation();
  
  print('\n=== Operations Complete ===');
}

// Synchronous operation - blocks execution
void syncOperation() {
  print('Start: Sync operation');
  
  // Simulate blocking work (BAD in Flutter)
  var sum = 0;
  for (var i = 0; i < 1000000000; i++) {
    sum += i;
  }
  
  print('End: Sync operation (sum: $sum)');
  // This blocks the thread - UI would freeze if this were in Flutter
}

// Asynchronous operation - doesn't block
void asyncOperation() async {
  print('Start: Async operation');
  
  // Simulate async work using Future.delayed
  await Future.delayed(Duration(seconds: 1));
  // The 'await' keyword pauses this function but doesn't block
  // Other code can continue executing
  
  var sum = 0;
  for (var i = 0; i < 1000000000; i++) {
    sum += i;
  }
  
  print('End: Async operation (sum: $sum)');
  // This runs later, after the delay
}
```

**Explanation:**

- **Synchronous operation**: The `syncOperation()` function executes completely before the next line of code runs. In Flutter, this would freeze the UI because it blocks the main thread.
- **Asynchronous operation**: The `asyncOperation()` function uses `await` to pause execution without blocking. The function yields control back to the event loop, allowing other code to run while waiting.
- **`Future.delayed()`**: Creates a Future that completes after a specified duration. This simulates an asynchronous operation like a network request.
- **`await` keyword**: Pauses the execution of an async function until the Future completes, but doesn't block the thread. Other code can execute during the wait.
- **Non-blocking**: Even though the heavy computation in `asyncOperation()` runs synchronously after the delay, the function itself is non-blocking because it uses `await` to yield control.
- **Flutter context**: In Flutter, always use asynchronous operations for:
  - Network requests
  - File I/O
  - Database operations
  - Long-running computations (use Isolates for these)
  - Animations and UI updates

### **Why Asynchronous Programming Matters in Flutter**

```dart
import 'dart:async';
import 'package:flutter/material.dart';

// Example of blocking vs. non-blocking UI

// BAD: Blocking operation (UI freezes)
void badExample(BuildContext context) {
  print('Loading data...');
  
  // Simulate a 2-second blocking operation
  var start = DateTime.now();
  while (DateTime.now().difference(start) < Duration(seconds: 2)) {
    // Busy wait - BLOCKS THE UI!
  }
  
  print('Data loaded');
  // During these 2 seconds, the UI is completely unresponsive
  // User can't interact with buttons, animations freeze
}

// GOOD: Non-blocking operation (UI remains responsive)
void goodExample(BuildContext context) async {
  print('Loading data...');
  
  // Simulate a 2-second non-blocking operation
  await Future.delayed(Duration(seconds: 2));
  // The UI remains responsive during these 2 seconds
  
  print('Data loaded');
}

// In a real Flutter widget
class DataFetchingWidget extends StatefulWidget {
  @override
  _DataFetchingWidgetState createState() => _DataFetchingWidgetState();
}

class _DataFetchingWidgetState extends State<DataFetchingWidget> {
  String _data = 'Not loaded';
  bool _isLoading = false;

  Future<void> fetchData() async {
    setState(() {
      _isLoading = true;
    });
    
    // Simulate API call
    await Future.delayed(Duration(seconds: 2));
    
    setState(() {
      _data = 'Data loaded successfully!';
      _isLoading = false;
    });
  }

  @override
  Widget build(BuildContext context) {
    return Column(
      children: [
        Text(_data),
        if (_isLoading)
          CircularProgressIndicator()
        else
          ElevatedButton(
            onPressed: _isLoading ? null : fetchData,
            child: Text('Load Data'),
          ),
      ],
    );
  }
}
```

**Explanation:**

- **Blocking UI**: The `badExample()` uses a `while` loop to simulate a blocking operation. This freezes the entire Flutter app - animations stop, buttons don't respond, and the user sees a frozen interface.
- **Non-blocking UI**: The `goodExample()` uses `Future.delayed()` with `await` to pause execution without blocking. The UI remains responsive throughout the 2-second wait.
- **`setState()`**: In Flutter, calling `setState()` rebuilds the widget tree. We use it to update the UI when loading state changes.
- **Loading indicator**: The `CircularProgressIndicator` shows while data is loading, providing visual feedback to the user.
- **Disabled button**: The button is disabled (`null` `onPressed`) while loading, preventing multiple simultaneous requests.
- **User experience**: Asynchronous programming is essential for providing a smooth, responsive user experience in Flutter applications. Users should never see a frozen UI.

---

## **6.2 Futures and the Event Loop**

Futures represent a value or error that will be available at some point in the future. They are the foundation of asynchronous programming in Dart.

### **Basic Future Creation**

```dart
void main() {
  print('1. Starting main()');
  
  // Create a Future that completes with a value
  Future<String> fetchUserData() {
    // Future constructor takes a function that returns a value
    return Future(() {
      print('   Inside Future - fetching user data');
      // This code runs asynchronously in the event loop
      return 'John Doe';
    });
  }
  
  // Using the Future
  var userFuture = fetchUserData();
  print('2. Future created (not completed yet)');
  
  // Register a callback when the Future completes
  userFuture.then((value) {
    print('3. Future completed with: $value');
    // This callback runs when the Future completes successfully
  });
  
  print('4. main() continues - Future is still pending');
  
  // Another example with Future.delayed
  print('\n5. Creating delayed Future');
  Future.delayed(Duration(seconds: 2), () {
    print('   Delayed Future completed after 2 seconds');
  });
  
  print('6. main() ends');
}

/*
Output:
1. Starting main()
   Inside Future - fetching user data
2. Future created (not completed yet)
4. main() continues - Future is still pending
5. Creating delayed Future
6. main() ends
3. Future completed with: John Doe
   Delayed Future completed after 2 seconds
*/
```

**Explanation:**

- **`Future` constructor**: Creates a Future that runs the given function asynchronously. The function's return value becomes the Future's result.
- **Execution timing**: The code inside `Future(() {...})` doesn't run immediately. It's scheduled to run in the next event loop iteration after synchronous code completes.
- **`then()` method**: Registers a callback that executes when the Future completes successfully. The callback receives the Future's result as a parameter.
- **Non-blocking**: After creating the Future, `main()` continues executing immediately. The Future runs later in the event loop.
- **Chaining**: You can chain multiple `then()` calls to process the result through multiple steps.
- **`Future.delayed()`**: Creates a Future that completes after a specified duration. Useful for simulating delays, retries, or timers.

### **Future Error Handling**

```dart
void main() {
  print('=== Successful Future ===');
  successfulFuture();
  
  print('\n=== Failed Future ===');
  failedFuture();
  
  print('\n=== Error Handling with catchError ===');
  errorHandledFuture();
  
  print('\n=== Complete with whenComplete ===');
  completeFuture();
}

// Future that completes successfully
Future<String> successfulFuture() {
  return Future.value('Success!');
  // Future.value creates a Future that's already completed
}

// Future that completes with an error
Future<String> failedFuture() {
  return Future.error(Exception('Something went wrong!'));
  // Future.error creates a Future that's already completed with an error
}

// Future with proper error handling
void errorHandledFuture() {
  Future<String> fetchData() {
    return Future(() {
      // Simulate an error condition
      if (DateTime.now().millisecond % 2 == 0) {
        throw Exception('Network error!');
      }
      return 'Data loaded';
    });
  }
  
  fetchData().then((data) {
    print('Success: $data');
  }).catchError((error) {
    print('Error caught: $error');
    // catchError handles errors from the Future
  });
  
  // Both success and error will be handled
}

// Future with cleanup using whenComplete
void completeFuture() {
  Future<String> loadData() {
    return Future.delayed(Duration(seconds: 1), () {
      return 'Data loaded';
    });
  }
  
  loadData()
      .then((data) {
        print('Data: $data');
      })
      .catchError((error) {
        print('Error: $error');
      })
      .whenComplete(() {
        print('Cleanup: Close connections, hide loading indicator');
        // whenComplete always runs, regardless of success or error
        // Useful for cleanup operations
      });
  
  print('Request initiated...');
}
```

**Explanation:**

- **`Future.value()`**: Creates a Future that's already completed with a value. Useful for creating already-resolved Futures.
- **`Future.error()`**: Creates a Future that's already completed with an error. Useful for testing or error conditions.
- **`catchError()`**: Registers a callback that handles errors from the Future. It catches any errors thrown in the Future or previous `then()` callbacks.
- **Error propagation**: If a Future throws an error (or if a `then()` callback throws), the error propagates down the chain until caught by `catchError()`.
- **`whenComplete()`**: Registers a callback that executes regardless of whether the Future succeeded or failed. It always runs after the Future completes.
- **Finally pattern**: `whenComplete()` is similar to `finally` blocks in try-catch statements. Use it for cleanup code that must run no matter what (closing resources, hiding loading spinners, etc.).
- **Error types**: `catchError()` can optionally accept an error test function to handle specific error types selectively.

### **Chaining Futures**

```dart
void main() {
  print('=== Chaining Futures ===');
  chainedOperations();
}

// Demonstrating Future chaining
Future<void> chainedOperations() {
  return Future(() {
    print('Step 1: User logged in');
    return 'user123';
  }).then((userId) {
    // This receives the result from Step 1
    print('Step 2: Fetching user profile for $userId');
    return fetchUserProfile(userId);
  }).then((profile) {
    // This receives the result from Step 2
    print('Step 3: Processing profile for ${profile['name']}');
    return processProfile(profile);
  }).then((processedProfile) {
    // This receives the result from Step 3
    print('Step 4: Displaying profile');
    displayProfile(processedProfile);
  }).catchError((error) {
    print('Error at any step: $error');
    // This catches errors from any step in the chain
  });
}

// Helper function that returns a Future
Future<Map<String, dynamic>> fetchUserProfile(String userId) {
  return Future(() {
    // Simulate API call
    return {
      'id': userId,
      'name': 'John Doe',
      'email': 'john@example.com',
    };
  });
}

// Helper function that returns a Future
Future<Map<String, dynamic>> processProfile(Map<String, dynamic> profile) {
  return Future(() {
    // Simulate data processing
    var processed = Map<String, dynamic>.from(profile);
    processed['processed'] = true;
    processed['timestamp'] = DateTime.now().toIso8601String();
    return processed;
  });
}

void displayProfile(Map<String, dynamic> profile) {
  print('Name: ${profile['name']}');
  print('Email: ${profile['email']}');
  print('Processed: ${profile['processed']}');
}

/*
Output:
Step 1: User logged in
Step 2: Fetching user profile for user123
Step 3: Processing profile for John Doe
Step 4: Displaying profile
Name: John Doe
Email: john@example.com
Processed: true
*/
```

**Explanation:**

- **Future chaining**: Each `then()` returns a new Future. You can chain multiple `then()` calls to sequence asynchronous operations.
- **Value passing**: The result from one `then()` callback is passed to the next callback in the chain. This creates a data flow through the chain.
- **Sequential execution**: Chained `then()` callbacks execute sequentially, one after the other, in the order they're chained.
- **Error propagation**: If any step in the chain throws an error, it propagates to the `catchError()` handler, skipping the remaining steps.
- **Return values**: Each `then()` callback can return a value or another Future. If it returns a Future, the chain waits for that Future to complete before continuing.
- **Use cases**: Chaining is perfect for:
  - Sequencing dependent operations (login → fetch data → process data)
  - Data transformation pipelines
  - Multi-step workflows
  - Request-response patterns

### **Multiple Futures with Future.wait**

```dart
void main() {
  print('=== Sequential Execution ===');
  sequentialExecution();
  
  print('\n=== Parallel Execution with Future.wait ===');
  parallelExecution();
  
  print('\n=== Future.wait with Error Handling ===');
  parallelWithError();
  
  print('\n=== Future.any - First to Complete ===');
  firstToComplete();
}

// Sequential: Operations run one after another
Future<void> sequentialExecution() async {
  var start = DateTime.now();
  
  // These run sequentially, one after another
  var result1 = await fetchData('API 1', Duration(seconds: 1));
  print('Result 1: $result1');
  
  var result2 = await fetchData('API 2', Duration(seconds: 1));
  print('Result 2: $result2');
  
  var result3 = await fetchData('API 3', Duration(seconds: 1));
  print('Result 3: $result3');
  
  var duration = DateTime.now().difference(start);
  print('Total time: ${duration.inSeconds} seconds (sequential)');
}

// Parallel: Operations run simultaneously
Future<void> parallelExecution() async {
  var start = DateTime.now();
  
  // These run in parallel
  var results = await Future.wait([
    fetchData('API 1', Duration(seconds: 1)),
    fetchData('API 2', Duration(seconds: 1)),
    fetchData('API 3', Duration(seconds: 1)),
  ]);
  // Future.wait waits for ALL Futures to complete
  
  print('Result 1: ${results[0]}');
  print('Result 2: ${results[1]}');
  print('Result 3: ${results[2]}');
  
  var duration = DateTime.now().difference(start);
  print('Total time: ${duration.inSeconds} seconds (parallel)');
}

// Future.wait with eagerError: false
Future<void> parallelWithError() async {
  var futures = <Future<String>>[
    fetchData('API 1', Duration(seconds: 1)),
    // This one fails
    Future.error(Exception('API 2 failed!')),
    fetchData('API 3', Duration(seconds: 1)),
  ];
  
  try {
    // eagerError: false waits for all Futures, even if some fail
    var results = await Future.wait(futures, eagerError: false);
    print('Results: $results');
  } catch (e) {
    print('Caught error: $e');
  }
}

// Future.any - returns the first Future to complete
Future<void> firstToComplete() async {
  var start = DateTime.now();
  
  // These run in parallel, but we only care about the first result
  var result = await Future.any([
    fetchData('Server 1', Duration(seconds: 3)),
    fetchData('Server 2', Duration(seconds: 2)),
    fetchData('Server 3', Duration(seconds: 1)),
  ]);
  
  var duration = DateTime.now().difference(start);
  print('Fastest result: $result (took ${duration.inSeconds} seconds)');
}

// Helper function to simulate data fetching
Future<String> fetchData(String api, Duration delay) {
  return Future.delayed(delay, () => 'Data from $api');
}

/*
Output for parallel execution:
=== Parallel Execution with Future.wait ===
Result 1: Data from API 1
Result 2: Data from API 2
Result 3: Data from API 3
Total time: 1 seconds (parallel)

Compare to sequential which took 3 seconds!
*/
```

**Explanation:**

- **Sequential execution**: Using multiple `await` statements runs operations one after another. The total time is the sum of all operation times (1 + 1 + 1 = 3 seconds).
- **`Future.wait()`**: Runs multiple Futures concurrently and waits for all of them to complete. The total time is approximately the duration of the longest operation (max(1, 1, 1) = 1 second).
- **Performance benefit**: `Future.wait()` can significantly improve performance when operations are independent and can run in parallel.
- **Result ordering**: `Future.wait()` returns a list of results in the same order as the input Futures, regardless of which completes first.
- **Error handling**:
  - Default (`eagerError: true`): If any Future fails, `Future.wait()` immediately throws an error, canceling remaining Futures.
  - `eagerError: false`: Waits for all Futures to complete, even if some fail. Returns a list containing errors for failed Futures.
- **`Future.any()`**: Returns the result of the first Future to complete, ignoring the others. Useful for:
  - Fetching data from multiple servers and using the fastest response
  - Timeout implementations
  - Race conditions where you only need the first result

---

## **6.3 Async and Await Patterns**

The `async` and `await` keywords provide a more readable way to write asynchronous code, making it look synchronous while maintaining non-blocking behavior.

### **Basic Async/Await Syntax**

```dart
void main() {
  print('=== Using .then() (Callback Style) ===');
  callbackStyle();
  
  print('\n=== Using async/await (Synchronous-looking Style) ===');
  asyncAwaitStyle();
}

// Callback style (before async/await)
void callbackStyle() {
  print('Step 1: Start');
  
  Future.delayed(Duration(seconds: 1)).then((_) {
    print('Step 2: After delay');
    return 'Result 1';
  }).then((result) {
    print('Step 3: Got $result');
    return Future.delayed(Duration(seconds: 1), () => 'Result 2');
  }).then((result) {
    print('Step 4: Got $result');
  }).catchError((error) {
    print('Error: $error');
  });
}

// async/await style
Future<void> asyncAwaitStyle() async {
  print('Step 1: Start');
  
  // await pauses execution until the Future completes
  await Future.delayed(Duration(seconds: 1));
  print('Step 2: After delay');
  
  var result1 = 'Result 1';
  print('Step 3: Got $result1');
  
  // await works with any Future
  var result2 = await Future.delayed(
    Duration(seconds: 1),
    () => 'Result 2',
  );
  print('Step 4: Got $result2');
}
```

**Explanation:**

- **`async` keyword**: Marks a function as asynchronous. An async function returns a Future automatically, even if it doesn't explicitly return a Future.
- **`await` keyword**: Pauses the execution of an async function until the awaited Future completes. The function yields control back to the event loop, allowing other code to run.
- **Synchronous appearance**: async/await makes asynchronous code look and read like synchronous code. The flow is linear and easy to follow, unlike nested callbacks.
- **No blocking**: Despite pausing with `await`, the function doesn't block the thread. Other code can execute during the wait.
- **Automatic Future wrapping**: If you return a value from an async function, it's automatically wrapped in a Future. The function signature effectively becomes `Future<ReturnType>`.
- **Best practice**: Prefer async/await over `.then()` for most cases, as it's more readable and easier to maintain.

### **Async Functions and Return Types**

```dart
void main() async {
  print('=== Return Value Examples ===');
  
  // Async function returning a value
  var value1 = await getValue();
  print('Got value: $value1');
  
  // Async function returning void
  await doSomething();
  print('Done with void function');
  
  // Async function with error handling
  try {
    var value2 = await getWithError();
    print('Got value: $value2');
  } catch (e) {
    print('Caught error: $e');
  }
  
  // Async function returning different types
  var user = await fetchUser();
  print('User: $user');
}

// Async function returning String
// The return type is Future<String> automatically
Future<String> getValue() async {
  await Future.delayed(Duration(seconds: 1));
  return 'Hello, World!';
  // This String is automatically wrapped in a Future
}

// Async function returning void
// The return type is Future<void>
Future<void> doSomething() async {
  print('Doing something...');
  await Future.delayed(Duration(seconds: 1));
  print('Something done!');
  // No explicit return needed for void
}

// Async function that throws an error
Future<String> getWithError() async {
  await Future.delayed(Duration(seconds: 1));
  throw Exception('Something went wrong!');
  // This error is wrapped in a failed Future
}

// Async function returning a complex type
Future<Map<String, dynamic>> fetchUser() async {
  await Future.delayed(Duration(seconds: 1));
  
  return {
    'id': 1,
    'name': 'John Doe',
    'email': 'john@example.com',
  };
}
```

**Explanation:**

- **`Future<ReturnType>`**: An async function that returns a value of type `T` has the return type `Future<T>`. Dart automatically wraps the return value in a Future.
- **`Future<void>`**: An async function that doesn't return a value has the return type `Future<void>`. The function completes when it finishes executing.
- **Automatic wrapping**: You don't need to explicitly create Futures in async functions. Just return a value, and Dart wraps it in a Future.
- **Error wrapping**: Throwing an error in an async function creates a Future that completes with that error. Callers can handle it with try-catch or `.catchError()`.
- **Complex return types**: Async functions can return any type, including custom classes, collections, and other Futures (though returning a Future from an async function is redundant).
- **Function signatures**: Always declare the return type as `Future<T>` for async functions. This makes the asynchronous nature explicit and helps with type checking.

### **Error Handling with Async/Await**

```dart
void main() async {
  print('=== Try-Catch-Finally in Async Functions ===');
  
  try {
    var result = await riskyOperation();
    print('Success: $result');
  } catch (e) {
    print('Caught error: $e');
  } finally {
    print('Cleanup in finally block');
  }
  
  print('\n=== Catching Specific Error Types ===');
  
  try {
    await specificErrorOperation();
  } on NetworkException catch (e) {
    print('Network error: ${e.message}');
  } on ValidationException catch (e) {
    print('Validation error: ${e.field}');
  } catch (e) {
    print('Other error: $e');
  }
  
  print('\n=== Rethrowing Errors ===');
  
  try {
    await rethrowOperation();
  } catch (e) {
    print('Caught: $e');
  }
}

// Async function that throws an error
Future<String> riskyOperation() async {
  await Future.delayed(Duration(seconds: 1));
  
  // Simulate an error
  if (DateTime.now().millisecond % 2 == 0) {
    throw Exception('Random error occurred!');
  }
  
  return 'Success!';
}

// Custom exception classes
class NetworkException implements Exception {
  final String message;
  NetworkException(this.message);
}

class ValidationException implements Exception {
  final String field;
  ValidationException(this.field);
}

// Async function with specific error types
Future<void> specificErrorOperation() async {
  await Future.delayed(Duration(seconds: 1));
  
  // Throw specific error based on condition
  var random = DateTime.now().millisecond % 3;
  switch (random) {
    case 0:
      throw NetworkException('Connection timeout');
    case 1:
      throw ValidationException('email');
    default:
      throw Exception('Unknown error');
  }
}

// Async function that catches and rethrows
Future<void> rethrowOperation() async {
  try {
    await riskyOperation();
  } catch (e) {
    print('Handling error locally: $e');
    // Do some local cleanup or logging
    rethrow; // Re-throw the error for the caller to handle
  }
}
```

**Explanation:**

- **Try-catch-finally**: Standard error handling works with async/await. `try` contains the code that might throw, `catch` handles errors, and `finally` runs cleanup code regardless of success or failure.
- **`on` keyword**: Use `on ErrorType` to catch specific error types. This is more precise than catching all errors with `catch`.
- **Exception handling flow**: When an error is thrown in an async function:
  1. The async function completes with that error
  2. The error propagates to the caller's try-catch block
  3. The `finally` block executes before error propagation
- **`rethrow`**: Use `rethrow` to re-throw a caught error after handling it locally. This allows local handling (logging, cleanup) while still notifying the caller.
- **Specific error types**: Creating custom exception classes helps with error handling and provides more context about what went wrong.
- **Best practices**:
  - Always handle errors appropriately (logging, user feedback, retry logic)
  - Use specific exception types when possible
  - Use `finally` for cleanup code (closing resources, hiding loading indicators)

### **Multiple Awaits and Concurrent Execution**

```dart
void main() async {
  print('=== Sequential Awaits ===');
  await sequentialAwaits();
  
  print('\n=== Parallel Awaits with Future.wait ===');
  await parallelAwaits();
  
  print('\n=== Fire and Forget (Don\'t Await) ===');
  fireAndForget();
  
  print('Main continues immediately');
  
  // Wait a bit to see fire-and-forget result
  await Future.delayed(Duration(seconds: 3));
}

// Sequential: Each await waits for the previous to complete
Future<void> sequentialAwaits() async {
  var start = DateTime.now();
  
  print('Starting task 1');
  var result1 = await task(1, Duration(seconds: 1));
  print('Task 1 complete: $result1');
  
  print('Starting task 2');
  var result2 = await task(2, Duration(seconds: 1));
  print('Task 2 complete: $result2');
  
  print('Starting task 3');
  var result3 = await task(3, Duration(seconds: 1));
  print('Task 3 complete: $result3');
  
  var duration = DateTime.now().difference(start);
  print('Total time: ${duration.inSeconds} seconds (sequential)');
}

// Parallel: All tasks run concurrently
Future<void> parallelAwaits() async {
  var start = DateTime.now();
  
  print('Starting all tasks');
  
  // Start all tasks concurrently
  var results = await Future.wait([
    task(1, Duration(seconds: 1)),
    task(2, Duration(seconds: 1)),
    task(3, Duration(seconds: 1)),
  ]);
  
  print('All tasks complete');
  print('Results: $results');
  
  var duration = DateTime.now().difference(start);
  print('Total time: ${duration.inSeconds} seconds (parallel)');
}

// Fire and forget: Start tasks but don't wait for them
void fireAndForget() {
  print('Starting background tasks');
  
  // Don't await - these run in the background
  backgroundTask('Email service', Duration(seconds: 2));
  backgroundTask('Analytics', Duration(seconds: 3));
  
  print('Background tasks started - continuing execution');
}

// Helper function that simulates work
Future<String> task(int id, Duration duration) {
  return Future.delayed(duration, () => 'Task $id result');
}

// Background task that doesn't need to be awaited
Future<void> backgroundTask(String name, Duration duration) async {
  print('  $name: Started');
  await Future.delayed(duration);
  print('  $name: Completed');
}

/*
Output for fire and forget:
=== Fire and Forget (Don't Await) ===
Starting background tasks
  Email service: Started
  Analytics: Started
Background tasks started - continuing execution
Main continues immediately

[2 seconds later]
  Email service: Completed

[3 seconds later]
  Analytics: Completed
*/
```

**Explanation:**

- **Sequential awaits**: Using multiple `await` statements one after another runs operations sequentially. Each operation completes before the next starts.
- **Parallel with `Future.wait()`**: To run operations in parallel, pass multiple Futures to `Future.wait()`. All operations start immediately, and the code waits for all to complete.
- **Performance difference**: Sequential execution takes the sum of all operation times. Parallel execution takes approximately the longest operation time.
- **Fire and forget**: Calling an async function without `await` starts it but doesn't wait for completion. The function runs in the background.
- **Use cases for fire and forget**:
  - Logging
  - Analytics
  - Background synchronization
  - Sending emails or notifications
- **Caution with fire and forget**:
  - Errors in fire-and-forget tasks won't be caught by the caller
  - You can't use the result of the task
  - The task may complete after the main function finishes
- **Best practice**: Only use fire and forget when:
  - You don't need the result
  - You don't need to handle errors
  - The task is truly independent

### **Real-World Example: API Data Fetching**

```dart
void main() async {
  print('=== Real-World API Example ===');
  
  try {
    // Simulate fetching user data from an API
    var userProfile = await fetchUserProfile('user123');
    print('User: ${userProfile.name}');
    
    // Sequential: Fetch posts, then comments
    print('\nSequential approach:');
    await sequentialFetch(userProfile.id);
    
    // Parallel: Fetch posts and comments simultaneously
    print('\nParallel approach:');
    await parallelFetch(userProfile.id);
    
  } catch (e) {
    print('Error: $e');
  }
}

// Simulated API response class
class UserProfile {
  final String id;
  final String name;
  final String email;
  
  UserProfile({required this.id, required this.name, required this.email});
  
  factory UserProfile.fromJson(Map<String, dynamic> json) {
    return UserProfile(
      id: json['id'],
      name: json['name'],
      email: json['email'],
    );
  }
}

// Fetch user profile from API
Future<UserProfile> fetchUserProfile(String userId) async {
  print('Fetching user profile for $userId...');
  
  // Simulate API delay
  await Future.delayed(Duration(milliseconds: 500));
  
  // Simulate API response
  return UserProfile.fromJson({
    'id': userId,
    'name': 'John Doe',
    'email': 'john@example.com',
  });
}

// Fetch posts sequentially (bad for performance)
Future<void> sequentialFetch(String userId) async {
  var start = DateTime.now();
  
  // Fetch posts
  var posts = await fetchPosts(userId);
  print('  Fetched ${posts.length} posts');
  
  // Then fetch comments for each post
  for (var post in posts) {
    var comments = await fetchComments(post.id);
    print('    Post "${post.title}" has ${comments.length} comments');
  }
  
  var duration = DateTime.now().difference(start);
  print('  Total time: ${duration.inMilliseconds}ms (sequential)');
}

// Fetch posts and comments in parallel (better performance)
Future<void> parallelFetch(String userId) async {
  var start = DateTime.now();
  
  // Fetch posts
  var posts = await fetchPosts(userId);
  print('  Fetched ${posts.length} posts');
  
  // Fetch all comments in parallel
  var commentFutures = posts.map((post) => fetchComments(post.id)).toList();
  var allComments = await Future.wait(commentFutures);
  
  for (var i = 0; i < posts.length; i++) {
    print('    Post "${posts[i].title}" has ${allComments[i].length} comments');
  }
  
  var duration = DateTime.now().difference(start);
  print('  Total time: ${duration.inMilliseconds}ms (parallel)');
}

// Simulated API calls
Future<List<Post>> fetchPosts(String userId) async {
  await Future.delayed(Duration(milliseconds: 300));
  
  return [
    Post(id: '1', userId: userId, title: 'First Post'),
    Post(id: '2', userId: userId, title: 'Second Post'),
    Post(id: '3', userId: userId, title: 'Third Post'),
  ];
}

Future<List<Comment>> fetchComments(String postId) async {
  await Future.delayed(Duration(milliseconds: 200));
  
  return [
    Comment(id: 'c1', postId: postId, text: 'Great post!'),
    Comment(id: 'c2', postId: postId, text: 'Thanks!'),
  ];
}

// Data models
class Post {
  final String id;
  final String userId;
  final String title;
  
  Post({required this.id, required this.userId, required this.title});
}

class Comment {
  final String id;
  final String postId;
  final String text;
  
  Comment({required this.id, required this.postId, required this.text});
}

/*
Output:
=== Real-World API Example ===
Fetching user profile for user123...
User: John Doe

Sequential approach:
  Fetched 3 posts
    Post "First Post" has 2 comments
    Post "Second Post" has 2 comments
    Post "Third Post" has 2 comments
  Total time: 900ms (sequential)

Parallel approach:
  Fetched 3 posts
    Post "First Post" has 2 comments
    Post "Second Post" has 2 comments
    Post "Third Post" has 2 comments
  Total time: 500ms (parallel)

Parallel is nearly 2x faster!
*/
```

**Explanation:**

- **Sequential approach**: Fetches posts, then for each post, fetches comments one at a time. Total time = posts time + (comments time × number of posts) = 300 + (200 × 3) = 900ms.
- **Parallel approach**: Fetches posts, then fetches all comments simultaneously using `Future.wait()`. Total time = max(posts time, comments time) = max(300, 200) = 500ms.
- **Performance gain**: Parallel execution is significantly faster when operations are independent.
- **`map()` with `Future.wait()`**: Uses `map()` to create a list of Future objects, then passes them to `Future.wait()` to execute them in parallel.
- **Real-world application**: This pattern is common in Flutter apps when:
  - Fetching data from multiple endpoints
  - Loading images
  - Processing independent data sets
- **Data models**: Defines simple classes to represent API response data. In a real app, these would be more complex and include serialization logic.
- **Best practice**: Always use parallel execution for independent operations to improve performance and user experience.

---

## **6.4 Working with Streams**

While Futures represent a single value that arrives asynchronously, Streams represent a sequence of asynchronous events over time. Streams are essential for handling continuous data flows like user input, file reading, or real-time updates.

### **Introduction to Streams**

```dart
void main() {
  print('=== Stream Basics ===');
  basicStream();
  
  print('\n=== Stream with Subscription ===');
  streamWithSubscription();
}

// Basic stream example
void basicStream() {
  // Create a stream that emits values every second
  var stream = Stream.periodic(Duration(seconds: 1), (count) {
    return 'Tick $count';
  });
  
  // Listen to the stream
  stream.listen((value) {
    print('Received: $value');
  });
  
  // The program will keep running because the stream is active
  // We need to wait a bit to see the output
  print('Stream listening started...');
}

// Stream with subscription control
void streamWithSubscription() {
  var stream = Stream.periodic(Duration(seconds: 1), (count) {
    return 'Count: $count';
  });
  
  // Subscribe and get a StreamSubscription
  var subscription = stream.listen(
    (value) {
      print('Data: $value');
    },
    onError: (error) {
      print('Error: $error');
    },
    onDone: () {
      print('Stream completed');
    },
  );
  
  // Cancel the subscription after 3 seconds
  Future.delayed(Duration(seconds: 3), () {
    print('Cancelling subscription...');
    subscription.cancel();
  });
}
```

**Explanation:**

- **`Stream`**: Represents a sequence of asynchronous events. Unlike a Future (single value), a Stream can emit multiple values over time.
- **`Stream.periodic()`**: Creates a stream that repeatedly emits values at a specified interval. The callback receives a count starting from 0.
- **`listen()`**: Subscribes to the stream and starts receiving events. The callback runs for each value emitted by the stream.
- **`StreamSubscription`**: Represents an active subscription to a stream. You can use it to control the subscription (pause, resume, cancel).
- **`onError` callback**: Called when the stream emits an error event.
- **`onDone` callback**: Called when the stream closes (no more events will be emitted).
- **`cancel()`**: Stops receiving events from the stream and releases resources. Important for preventing memory leaks.
- **Continuous operation**: Unlike Futures, streams can run indefinitely. You must explicitly cancel subscriptions when done.

### **Stream Creation Methods**

```dart
void main() async {
  print('=== Stream.fromIterable ===');
  await fromIterableExample();
  
  print('\n=== Stream.fromFuture ===');
  await fromFutureExample();
  
  print('\n=== Stream.value ===');
  await valueExample();
  
  print('\n=== Custom Stream with StreamController ===');
  await customStreamExample();
}

// Create stream from iterable
Future<void> fromIterableExample() async {
  var numbers = [1, 2, 3, 4, 5];
  var stream = Stream.fromIterable(numbers);
  
  await for (var number in stream) {
    print('Number: $number');
  }
  
  print('Stream completed');
}

// Create stream from Future
Future<void> fromFutureExample() async {
  var future = Future.delayed(Duration(seconds: 1), () => 'Hello');
  var stream = Stream.fromFuture(future);
  
  await for (var value in stream) {
    print('From Future: $value');
  }
  
  print('Stream completed');
}

// Create stream with single value
Future<void> valueExample() async {
  var stream = Stream.value(42);
  
  await for (var value in stream) {
    print('Value: $value');
  }
  
  print('Stream completed');
}

// Create custom stream with StreamController
Future<void> customStreamExample() async {
  var controller = StreamController<String>();
  
  // Start adding values to the stream
  Future.delayed(Duration(milliseconds: 500), () {
    controller.add('First value');
  });
  
  Future.delayed(Duration(milliseconds: 1000), () {
    controller.add('Second value');
  });
  
  Future.delayed(Duration(milliseconds: 1500), () {
    controller.add('Third value');
    controller.close(); // Close the stream
  });
  
  // Listen to the stream
  await for (var value in controller.stream) {
    print('Received: $value');
  }
  
  print('Stream completed');
  
  // Don't forget to close the controller!
  controller.close();
}
```

**Explanation:**

- **`Stream.fromIterable()`**: Creates a stream that emits all values from an iterable (like a List) in order, then closes.
- **`Stream.fromFuture()`**: Creates a stream that emits a single value (the Future's result) when the Future completes, then closes.
- **`Stream.value()`**: Creates a stream that immediately emits a single value, then closes. Equivalent to `Stream.fromFuture(Future.value(value))`.
- **`StreamController`**: Allows you to create and control a custom stream. You can add values, errors, and close the stream programmatically.
- **`add()` method**: Adds a value to the stream. All listeners receive this value.
- **`close()` method**: Closes the stream, signaling that no more values will be emitted. The `onDone` callback fires after this.
- **`await for` loop**: A convenient way to iterate over stream values asynchronously. It waits for each value to arrive, then processes it.
- **Use cases**:
  - `fromIterable`: Processing existing collections asynchronously
  - `fromFuture`: Wrapping a single async operation in a stream
  - `StreamController`: Creating custom streams with complex logic
  - `value`: Testing or scenarios needing a single stream event

### **Stream Transformations**

```dart
void main() async {
  print('=== Stream.map ===');
  await mapExample();
  
  print('\n=== Stream.where ===');
  await whereExample();
  
  print('\n=== Stream.take and skip ===');
  await takeSkipExample();
  
  print('\n=== Stream.transform ===');
  await transformExample();
  
  print('\n=== Chaining Transformations ===');
  await chainTransformations();
}

// map: Transform each value
Future<void> mapExample() async {
  var numbers = Stream.fromIterable([1, 2, 3, 4, 5]);
  
  var doubled = numbers.map((n) => n * 2);
  
  await for (var value in doubled) {
    print('Doubled: $value');
  }
}

// where: Filter values
Future<void> whereExample() async {
  var numbers = Stream.fromIterable([1, 2, 3, 4, 5, 6, 7, 8, 9, 10]);
  
  var evens = numbers.where((n) => n % 2 == 0);
  
  await for (var value in evens) {
    print('Even: $value');
  }
}

// take: Get first n values, skip: Skip first n values
Future<void> takeSkipExample() async {
  var numbers = Stream.periodic(Duration(milliseconds: 200), (count) => count);
  
  // Take first 5 values
  var firstFive = numbers.take(5);
  
  print('First 5:');
  await for (var value in firstFive) {
    print('  $value');
  }
  
  print('\nSkipping first 3, taking next 5:');
  
  // Create a new stream
  var numbers2 = Stream.periodic(Duration(milliseconds: 200), (count) => count);
  var skipAndTake = numbers2.skip(3).take(5);
  
  await for (var value in skipAndTake) {
    print('  $value');
  }
}

// transform: Use StreamTransformer
Future<void> transformExample() async {
  var numbers = Stream.fromIterable([1, 2, 3, 4, 5]);
  
  // Create a transformer that doubles and converts to string
  var transformer = StreamTransformer<int, String>.fromHandlers(
    handleData: (data, sink) {
      sink.add('Value: ${data * 2}');
    },
  );
  
  var transformed = numbers.transform(transformer);
  
  await for (var value in transformed) {
    print(value);
  }
}

// Chain multiple transformations
Future<void> chainTransformations() async {
  var numbers = Stream.periodic(Duration(milliseconds: 100), (count) => count);
  
  var result = numbers
      .take(10)           // Get first 10 values
      .where((n) => n.isEven)  // Filter evens
      .map((n) => n * n)   // Square each
      .map((n) => 'Square of ${n ~/ 2} is $n'); // Format as string
  
  await for (var value in result) {
    print(value);
  }
}
```

**Explanation:**

- **`map()`**: Transforms each value emitted by the stream using a function. Creates a new stream with transformed values.
- **`where()`**: Filters values, creating a new stream that only emits values that pass the predicate function (return `true`).
- **`take()`**: Takes only the first `n` values from the stream, then closes the new stream.
- **`skip()`**: Skips the first `n` values from the stream, emitting values after that.
- **`transform()`**: Applies a `StreamTransformer` to the stream. Transformers can handle data, errors, and done events with custom logic.
- **`StreamTransformer`**: A powerful way to create custom stream transformations. Can handle complex transformation logic.
- **Chaining transformations**: Multiple transformations can be chained together, with each transformation operating on the result of the previous one.
- **Lazy evaluation**: Stream transformations are lazy. They don't start processing until you listen to the final stream. Each value flows through the entire chain.
- **Use cases**:
  - `map`: Data conversion, formatting
  - `where`: Filtering, validation
  - `take`/`skip`: Pagination, limiting results
  - `transform`: Complex custom transformations

### **Advanced Stream Operations**

```dart
void main() async {
  print('=== Stream.expand ===');
  await expandExample();
  
  print('\n=== Stream.asyncMap ===');
  await asyncMapExample();
  
  print('\n=== Stream.asyncExpand ===');
  await asyncExpandExample();
  
  print('\n=== Stream.merge ===');
  await mergeExample();
  
  print('\n=== Stream.zip ===');
  await zipExample();
}

// expand: Transform each value into multiple values
Future<void> expandExample() async {
  var words = Stream.fromIterable(['hello', 'world']);
  
  // Expand each word into its characters
  var characters = words.expand((word) => word.split(''));
  
  await for (var char in characters) {
    print('Character: $char');
  }
}

// asyncMap: Transform each value asynchronously
Future<void> asyncMapExample() async {
  var ids = Stream.fromIterable([1, 2, 3, 4, 5]);
  
  // Fetch data for each ID asynchronously
  var users = ids.asyncMap((id) => fetchUser(id));
  
  await for (var user in users) {
    print('User: $user');
  }
}

// asyncExpand: Transform each value into multiple values asynchronously
Future<void> asyncExpandExample() async {
  var userIds = Stream.fromIterable(['u1', 'u2']);
  
  // For each user, fetch their posts asynchronously
  var allPosts = userIds.asyncExpand((userId) => fetchPosts(userId));
  
  await for (var post in allPosts) {
    print('Post: $post');
  }
}

// merge: Combine multiple streams
Future<void> mergeExample() async {
  var stream1 = Stream.periodic(Duration(milliseconds: 300), (count) => 'A$count');
  var stream2 = Stream.periodic(Duration(milliseconds: 500), (count) => 'B$count');
  
  var merged = StreamGroup.merge([stream1, stream2]);
  
  // Take 5 values and then cancel
  var subscription = merged.take(5).listen((value) {
    print('Merged: $value');
  });
  
  await subscription.asFuture();
  subscription.cancel();
}

// zip: Combine values from multiple streams
Future<void> zipExample() async {
  var names = Stream.fromIterable(['Alice', 'Bob', 'Charlie']);
  var ages = Stream.fromIterable([25, 30, 35]);
  
  // Combine name and age pairs
  var zipped = Stream.zip(names, ages, (name, age) => '$name is $age');
  
  await for (var info in zipped) {
    print(info);
  }
}

// Helper async functions
Future<String> fetchUser(int id) async {
  await Future.delayed(Duration(milliseconds: 200));
  return 'User $id';
}

Future<List<String>> fetchPosts(String userId) async {
  await Future.delayed(Duration(milliseconds: 300));
  return ['$userId Post 1', '$userId Post 2'];
}
```

**Explanation:**

- **`expand()`**: Transforms each value into multiple values. The callback returns an iterable, and each element becomes a separate value in the new stream.
- **`asyncMap()`**: Like `map()`, but the transformation function is asynchronous (returns a Future). Each value is transformed one at a time, waiting for each async operation to complete.
- **`asyncExpand()`**: Like `expand()`, but the expansion function is asynchronous (returns a Future of an iterable). Useful for expanding each value into multiple values fetched asynchronously.
- **`StreamGroup.merge()`**: Combines multiple streams into one. Values from all streams are emitted in the order they arrive, interleaved.
- **`Stream.zip()`**: Combines two streams by pairing values. Each pair consists of the nth value from each stream. The combined stream closes when either input stream closes.
- **Execution timing**:
  - `asyncMap`: Sequential - waits for each transformation to complete before starting the next
  - `asyncExpand`: Sequential - waits for each expansion to complete before starting the next
  - `merge`: Parallel - values arrive as soon as any source stream emits
  - `zip`: Parallel - pairs values as they arrive from both streams
- **`StreamGroup`**: A utility class from `dart:async` for working with multiple streams. Requires importing `package:async/async.dart` in real applications.
- **Use cases**:
  - `expand`: Flattening nested structures
  - `asyncMap`: API calls for each item
  - `asyncExpand`: Fetching multiple related items per source item
  - `merge`: Combining data from multiple sources
  - `zip`: Combining related data streams

---

## **6.5 StreamBuilder in Flutter**

`StreamBuilder` is a Flutter widget that builds itself based on the latest snapshot of interaction with a Stream. It's essential for creating reactive UIs that update automatically when data arrives.

### **Basic StreamBuilder Usage**

```dart
import 'dart:async';
import 'package:flutter/material.dart';

// Basic StreamBuilder example
class BasicStreamBuilderExample extends StatelessWidget {
  // Create a stream that emits a number every second
  Stream<int> get _numberStream {
    return Stream.periodic(Duration(seconds: 1), (count) => count).take(10);
  }

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(title: Text('Basic StreamBuilder')),
      body: Center(
        child: StreamBuilder<int>(
          stream: _numberStream,
          initialData: 0,
          builder: (context, snapshot) {
            // The builder is called every time the stream emits a value
            
            if (snapshot.hasError) {
              // Handle error state
              return Text(
                'Error: ${snapshot.error}',
                style: TextStyle(color: Colors.red),
              );
            }
            
            if (!snapshot.hasData) {
              // Show loading while waiting for first value
              return CircularProgressIndicator();
            }
            
            // Show the data
            return Column(
              mainAxisAlignment: MainAxisAlignment.center,
              children: [
                Text(
                  'Count:',
                  style: TextStyle(fontSize: 20),
                ),
                Text(
                  '${snapshot.data}',
                  style: TextStyle(
                    fontSize: 72,
                    fontWeight: FontWeight.bold,
                  ),
                ),
                Text(
                  'Connection state: ${snapshot.connectionState}',
                  style: TextStyle(color: Colors.grey),
                ),
              ],
            );
          },
        ),
      ),
    );
  }
}
```

**Explanation:**

- **`StreamBuilder<T>`**: A generic widget that listens to a stream of type `T` and rebuilds when new data arrives.
- **`stream` parameter**: The stream to listen to. The widget subscribes to this stream automatically and handles subscription lifecycle.
- **`initialData` parameter**: Optional initial data to display before the first stream value arrives. Prevents showing a loading state initially.
- **`builder` callback**: Called whenever the stream emits a new value or changes state. Receives the `BuildContext` and an `AsyncSnapshot<T>`.
- **`AsyncSnapshot<T>`**: Contains the current state of the stream:
  - `hasData`: Whether a value is available
  - `data`: The current value (if `hasData` is true)
  - `hasError`: Whether an error occurred
  - `error`: The error object (if `hasError` is true)
  - `connectionState`: The current state of the stream connection
- **`ConnectionState` enum**: Can be:
  - `none`: Not connected to any stream
  - `waiting`: Connected and waiting for first event
  - `active`: Connected and receiving events
  - `done`: Connected and stream has closed
- **Lifecycle management**: `StreamBuilder` automatically subscribes to the stream when created and unsubscribes when disposed, preventing memory leaks.
- **Rebuild behavior**: The `builder` callback is called for every stream event. Ensure the builder is efficient to avoid performance issues.

### **StreamBuilder with Loading and Error States**

```dart
import 'dart:async';
import 'package:flutter/material.dart';

// StreamBuilder with comprehensive state handling
class StatefulStreamBuilderExample extends StatelessWidget {
  // Stream that simulates API call with possible errors
  Stream<String> fetchData() {
    return Stream.periodic(Duration(seconds: 2), (count) {
      // Simulate error on count 3
      if (count == 3) {
        throw Exception('Network error occurred!');
      }
      // Simulate completion after 5 values
      if (count >= 5) {
        throw Exception('Stream closed');
      }
      return 'Data item ${count + 1}';
    });
  }

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(title: Text('Stateful StreamBuilder')),
      body: Center(
        child: StreamBuilder<String>(
          stream: fetchData(),
          builder: (context, snapshot) {
            // Handle different connection states
            switch (snapshot.connectionState) {
              case ConnectionState.waiting:
                // Initial loading state
                return _buildLoading();
              
              case ConnectionState.active:
                // Stream is active and emitting values
                if (snapshot.hasError) {
                  return _buildError(snapshot.error!);
                }
                if (snapshot.hasData) {
                  return _buildData(snapshot.data!);
                }
                return _buildLoading();
              
              case ConnectionState.done:
                // Stream has completed
                if (snapshot.hasError) {
                  // Stream ended with error
                  return _buildError(snapshot.error!, isFinal: true);
                }
                return _buildComplete();
              
              case ConnectionState.none:
                // No stream connected
                return _buildNoConnection();
            }
          },
        ),
      ),
    );
  }

  Widget _buildLoading() {
    return Column(
      mainAxisAlignment: MainAxisAlignment.center,
      children: [
        CircularProgressIndicator(),
        SizedBox(height: 16),
        Text('Loading data...'),
      ],
    );
  }

  Widget _buildError(Object error, {bool isFinal = false}) {
    return Column(
      mainAxisAlignment: MainAxisAlignment.center,
      children: [
        Icon(
          Icons.error_outline,
          size: 64,
          color: Colors.red,
        ),
        SizedBox(height: 16),
        Text(
          'Error occurred',
          style: TextStyle(fontSize: 20, fontWeight: FontWeight.bold),
        ),
        SizedBox(height: 8),
        Text(
          error.toString(),
          style: TextStyle(color: Colors.grey),
          textAlign: TextAlign.center,
        ),
        if (isFinal) ...[
          SizedBox(height: 16),
          Text(
            'Stream ended',
            style: TextStyle(color: Colors.orange),
          ),
        ],
      ],
    );
  }

  Widget _buildData(String data) {
    return Card(
      elevation: 4,
      child: Padding(
        padding: EdgeInsets.all(24),
        child: Column(
          mainAxisSize: MainAxisSize.min,
          children: [
            Icon(
              Icons.check_circle_outline,
              size: 48,
              color: Colors.green,
            ),
            SizedBox(height: 16),
            Text(
              data,
              style: TextStyle(fontSize: 24, fontWeight: FontWeight.bold),
            ),
          ],
        ),
      ),
    );
  }

  Widget _buildComplete() {
    return Column(
      mainAxisAlignment: MainAxisAlignment.center,
      children: [
        Icon(
          Icons.done_all,
          size: 64,
          color: Colors.green,
        ),
        SizedBox(height: 16),
        Text(
          'All data loaded',
          style: TextStyle(fontSize: 20, fontWeight: FontWeight.bold),
        ),
      ],
    );
  }

  Widget _buildNoConnection() {
    return Column(
      mainAxisAlignment: MainAxisAlignment.center,
      children: [
        Icon(
          Icons.warning,
          size: 64,
          color: Colors.orange,
        ),
        SizedBox(height: 16),
        Text(
          'No stream connected',
          style: TextStyle(fontSize: 20),
        ),
      ],
    );
  }
}
```

**Explanation:**

- **Comprehensive state handling**: This example handles all possible states of a stream connection and data availability.
- **`connectionState` switching**: Using a switch statement on `snapshot.connectionState` provides a clear structure for handling different states.
- **State breakdown**:
  - `waiting`: Initial state before first event. Good for showing initial loading indicator.
  - `active`: Stream is emitting events. Check for errors and data within this state.
  - `done`: Stream has closed. Distinguish between successful completion and error-based completion.
  - `none`: No stream is connected (edge case, rarely used).
- **`hasError` within states**: Even in `active` state, check `hasError` to handle errors that occur during stream execution.
- **Final errors**: When `connectionState` is `done` and `hasError` is true, the stream ended with an error.
- **Modular widget methods**: Separating each state's UI into its own method makes the code more readable and maintainable.
- **User feedback**: Each state provides clear visual feedback to the user about what's happening.
- **Real-world application**: This pattern is essential for API calls, data synchronization, and any operation that can fail or take time.

### **Real-World Example: Chat Application**

```dart
import 'dart:async';
import 'package:flutter/material.dart';

// Chat message model
class ChatMessage {
  final String id;
  final String text;
  final String senderId;
  final DateTime timestamp;
  
  ChatMessage({
    required this.id,
    required this.text,
    required this.senderId,
    required this.timestamp,
  });
}

// Chat service simulating backend
class ChatService {
  // Stream of chat messages
  Stream<ChatMessage> getMessages(String chatId) {
    // Simulate receiving messages from a websocket
    return Stream.periodic(Duration(seconds: 2), (count) {
      return ChatMessage(
        id: 'msg_$count',
        text: _getRandomMessage(count),
        senderId: count % 2 == 0 ? 'user1' : 'user2',
        timestamp: DateTime.now(),
      );
    });
  }
  
  String _getRandomMessage(int count) {
    final messages = [
      'Hello!',
      'How are you?',
      'What are you doing?',
      'That\'s interesting!',
      'I agree.',
      'Let me think about it.',
      'Sure, why not?',
      'Sounds good to me.',
    ];
    return messages[count % messages.length];
  }
}

// Chat screen with StreamBuilder
class ChatScreen extends StatefulWidget {
  final String chatId;
  
  ChatScreen({required this.chatId});
  
  @override
  _ChatScreenState createState() => _ChatScreenState();
}

class _ChatScreenState extends State<ChatScreen> {
  final ChatService _chatService = ChatService();
  final TextEditingController _messageController = TextEditingController();
  final ScrollController _scrollController = ScrollController();
  
  late Stream<ChatMessage> _messageStream;
  List<ChatMessage> _messages = [];

  @override
  void initState() {
    super.initState();
    _messageStream = _chatService.getMessages(widget.chatId);
  }

  @override
  void dispose() {
    _messageController.dispose();
    _scrollController.dispose();
    super.dispose();
  }

  void _sendMessage() {
    final text = _messageController.text.trim();
    if (text.isEmpty) return;
    
    // In a real app, this would send to the backend
    // For demo, we just clear the input
    _messageController.clear();
    
    // Simulate adding message
    setState(() {
      _messages.add(ChatMessage(
        id: DateTime.now().toString(),
        text: text,
        senderId: 'current_user',
        timestamp: DateTime.now(),
      ));
    });
  }

  void _scrollToBottom() {
    if (_scrollController.hasClients) {
      _scrollController.animateTo(
        _scrollController.position.maxScrollExtent,
        duration: Duration(milliseconds: 300),
        curve: Curves.easeOut,
      );
    }
  }

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        title: Text('Chat'),
        actions: [
          StreamBuilder<ChatMessage>(
            stream: _messageStream,
            builder: (context, snapshot) {
              if (snapshot.hasData) {
                return Padding(
                  padding: EdgeInsets.only(right: 16),
                  child: Icon(
                    Icons.circle,
                    color: Colors.green,
                    size: 12,
                  ),
                );
              }
              return SizedBox.shrink();
            },
          ),
        ],
      ),
      body: Column(
        children: [
          // Messages list with StreamBuilder
          Expanded(
            child: StreamBuilder<ChatMessage>(
              stream: _messageStream,
              initialData: null,
              builder: (context, snapshot) {
                if (snapshot.hasError) {
                  return _buildErrorState(snapshot.error!);
                }
                
                if (snapshot.hasData) {
                  // Add new message to list
                  final newMessage = snapshot.data!;
                  if (!_messages.any((m) => m.id == newMessage.id)) {
                    _messages.add(newMessage);
                    // Auto-scroll to bottom when new message arrives
                    WidgetsBinding.instance.addPostFrameCallback((_) {
                      _scrollToBottom();
                    });
                  }
                }
                
                if (_messages.isEmpty) {
                  return _buildEmptyState();
                }
                
                return _buildMessageList();
              },
            ),
          ),
          
          // Message input
          _buildMessageInput(),
        ],
      ),
    );
  }

  Widget _buildMessageList() {
    return ListView.builder(
      controller: _scrollController,
      padding: EdgeInsets.all(16),
      itemCount: _messages.length,
      itemBuilder: (context, index) {
        final message = _messages[index];
        final isCurrentUser = message.senderId == 'current_user';
        
        return Align(
          alignment: isCurrentUser ? Alignment.centerRight : Alignment.centerLeft,
          child: Container(
            margin: EdgeInsets.only(bottom: 8),
            padding: EdgeInsets.all(12),
            decoration: BoxDecoration(
              color: isCurrentUser ? Colors.blue : Colors.grey[300],
              borderRadius: BorderRadius.circular(8),
            ),
            child: Column(
              crossAxisAlignment: CrossAxisAlignment.start,
              mainAxisSize: MainAxisSize.min,
              children: [
                Text(
                  message.text,
                  style: TextStyle(
                    color: isCurrentUser ? Colors.white : Colors.black,
                  ),
                ),
                SizedBox(height: 4),
                Text(
                  _formatTime(message.timestamp),
                  style: TextStyle(
                    fontSize: 10,
                    color: isCurrentUser 
                      ? Colors.white70 
                      : Colors.black54,
                  ),
                ),
              ],
            ),
          ),
        );
      },
    );
  }

  Widget _buildMessageInput() {
    return Container(
      padding: EdgeInsets.all(16),
      decoration: BoxDecoration(
        color: Colors.white,
        boxShadow: [
          BoxShadow(
            blurRadius: 10,
            color: Colors.black.withOpacity(0.1),
          ),
        ],
      ),
      child: Row(
        children: [
          Expanded(
            child: TextField(
              controller: _messageController,
              decoration: InputDecoration(
                hintText: 'Type a message...',
                border: OutlineInputBorder(),
                contentPadding: EdgeInsets.symmetric(
                  horizontal: 16,
                  vertical: 8,
                ),
              ),
            ),
          ),
          SizedBox(width: 8),
          IconButton(
            icon: Icon(Icons.send),
            onPressed: _sendMessage,
          ),
        ],
      ),
    );
  }

  Widget _buildEmptyState() {
    return Center(
      child: Column(
        mainAxisAlignment: MainAxisAlignment.center,
        children: [
          Icon(
            Icons.chat_bubble_outline,
            size: 64,
            color: Colors.grey,
          ),
          SizedBox(height: 16),
          Text(
            'No messages yet',
            style: TextStyle(fontSize: 18, color: Colors.grey),
          ),
          SizedBox(height: 8),
          Text(
            'Start the conversation!',
            style: TextStyle(color: Colors.grey),
          ),
        ],
      ),
    );
  }

  Widget _buildErrorState(Object error) {
    return Center(
      child: Column(
        mainAxisAlignment: MainAxisAlignment.center,
        children: [
          Icon(
            Icons.error_outline,
            size: 64,
            color: Colors.red,
          ),
          SizedBox(height: 16),
          Text(
            'Connection error',
            style: TextStyle(fontSize: 18, color: Colors.red),
          ),
          SizedBox(height: 8),
          ElevatedButton(
            onPressed: () {
              setState(() {
                _messages.clear();
              });
            },
            child: Text('Retry'),
          ),
        ],
      ),
    );
  }

  String _formatTime(DateTime timestamp) {
    final now = DateTime.now();
    final difference = now.difference(timestamp);
    
    if (difference.inMinutes < 1) {
      return 'Just now';
    } else if (difference.inHours < 1) {
      return '${difference.inMinutes}m ago';
    } else if (difference.inDays < 1) {
      return '${difference.inHours}h ago';
    } else {
      return '${difference.inDays}d ago';
    }
  }
}
```

**Explanation:**

- **Real-time chat UI**: This example demonstrates a complete chat interface using `StreamBuilder` to receive messages in real-time.
- **Message accumulation**: Messages from the stream are accumulated in a list (`_messages`). This is necessary because `StreamBuilder` only provides the latest value, not the entire history.
- **Auto-scroll**: When a new message arrives, the chat auto-scrolls to the bottom using `ScrollController` and `addPostFrameCallback`.
- **Connection indicator**: A small green dot in the app bar shows when the connection is active using a separate `StreamBuilder`.
- **Message bubbles**: Messages are displayed as bubbles with different colors for the current user vs. other users.
- **Input field**: Users can type and send messages. In a real app, this would send to a backend via an API or websocket.
- **State management**: The widget maintains state for messages, scroll position, and input text.
- **Error handling**: Displays an error state if the stream fails and provides a retry button.
- **Empty state**: Shows a friendly message when there are no messages yet.
- **Timestamp formatting**: Helper function formats timestamps relative to the current time (e.g., "2m ago").
- **Best practices**:
  - Separate UI components into helper methods
  - Dispose controllers to prevent memory leaks
  - Provide visual feedback for all states
  - Handle connection errors gracefully
  - Auto-scroll to show new content

---

## **6.6 StreamController and Custom Streams**

For more complex scenarios, you can create custom streams using `StreamController`. This gives you full control over when values, errors, and done events are emitted.

### **Creating Custom Streams**

```dart
import 'dart:async';

void main() async {
  print('=== Basic StreamController ===');
  await basicStreamController();
  
  print('\n=== StreamController with Error Handling ===');
  await errorHandlingController();
  
  print('\n=== StreamController with Broadcast Stream ===');
  await broadcastStreamController();
}

// Basic StreamController
Future<void> basicStreamController() async {
  var controller = StreamController<String>();
  
  // Listen to the stream
  var subscription = controller.stream.listen(
    (value) => print('Received: $value'),
    onError: (error) => print('Error: $error'),
    onDone: () => print('Stream completed'),
  );
  
  // Add values to the stream
  controller.add('First');
  controller.add('Second');
  controller.add('Third');
  
  // Close the stream
  controller.close();
  
  // Wait for the stream to complete
  await subscription.asFuture();
  
  // Dispose the controller
  await controller.close();
}

// StreamController with error handling
Future<void> errorHandlingController() async {
  var controller = StreamController<int>();
  
  controller.stream.listen(
    (value) => print('Value: $value'),
    onError: (error) => print('Caught error: $error'),
    onDone: () => print('Stream done'),
    cancelOnError: false, // Don't cancel on error
  );
  
  // Add values
  controller.add(1);
  controller.add(2);
  
  // Add an error
  controller.addError(Exception('Something went wrong!'));
  
  // Stream continues after error (because cancelOnError: false)
  controller.add(3);
  
  controller.close();
  await controller.close();
}

// Broadcast stream (multiple listeners)
Future<void> broadcastStreamController() async {
  var controller = StreamController<String>.broadcast();
  
  // First listener
  var sub1 = controller.stream.listen((value) {
    print('Listener 1: $value');
  });
  
  // Second listener
  var sub2 = controller.stream.listen((value) {
    print('Listener 2: $value');
  });
  
  // Add values
  controller.add('Hello');
  controller.add('World');
  
  controller.close();
  
  await Future.wait([sub1.asFuture(), sub2.asFuture()]);
  await controller.close();
}
```

**Explanation:**

- **`StreamController`**: A class that allows you to create and control a stream programmatically. You can add values, errors, and close the stream at will.
- **`add()` method**: Emits a value to all listeners of the stream. Listeners receive this value immediately.
- **`addError()` method**: Emits an error event to all listeners. The error flows through the stream like any other event.
- **`close()` method**: Signals that no more events will be emitted. Triggers the `onDone` callback for all listeners.
- **`stream` property**: The actual stream that listeners subscribe to. The controller manages this stream.
- **`cancelOnError` parameter**: Determines whether the subscription is canceled when an error is received. `false` allows the stream to continue after errors.
- **Single-subscription vs. broadcast**:
  - Single-subscription (default): Only one listener allowed. Good for one-to-one streams like API responses.
  - Broadcast: Multiple listeners allowed. Good for one-to-many streams like events.
- **`StreamController.broadcast()`**: Creates a broadcast controller that allows multiple listeners.
- **Cleanup**: Always close controllers when done to prevent memory leaks. `await controller.close()` ensures all events are processed before cleanup.

### **StreamController with Sink**

```dart
import 'dart:async';

void main() async {
  print('=== StreamController with Sink ===');
  await sinkExample();
  
  print('\n=== StreamController with Transformations ===');
  await transformationExample();
}

// Using StreamSink for more control
Future<void> sinkExample() async {
  var controller = StreamController<String>();
  
  // Get the sink
  var sink = controller.sink;
  
  // Listen to the stream
  controller.stream.listen(
    (value) => print('Received: $value'),
    onDone: () => print('Sink closed'),
  );
  
  // Add values using sink
  sink.add('Value 1');
  sink.add('Value 2');
  
  // Add error using sink
  sink.addError(Exception('Sink error'));
  
  // Close using sink
  sink.close();
  
  await controller.close();
}

// StreamController with transformations
Future<void> transformationExample() async {
  var controller = StreamController<int>();
  
  // Transform the stream
  var doubledStream = controller.stream
      .where((n) => n.isEven)
      .map((n) => n * 2)
      .take(5);
  
  // Listen to transformed stream
  var subscription = doubledStream.listen(
    (value) => print('Doubled: $value'),
    onDone: () => print('Transformation complete'),
  );
  
  // Add values to the original stream
  for (var i = 0; i < 10; i++) {
    controller.add(i);
  }
  
  controller.close();
  await subscription.asFuture();
  await controller.close();
}
```

**Explanation:**

- **`StreamSink`**: The `sink` property of a `StreamController` provides methods to add events to the stream. It's the same as calling methods directly on the controller.
- **Sink methods**: `add()`, `addError()`, and `close()` are available on the sink. Using the sink can be more convenient when you need to pass the event-emitting interface to other code.
- **Transforming streams**: You can apply transformations to the controller's stream using `map`, `where`, `take`, etc. Listeners subscribe to the transformed stream, not the original.
- **Chaining transformations**: Multiple transformations can be chained. Values flow through each transformation in order.
- **Independence**: Transformations don't affect the original stream. You can have multiple transformed streams from the same controller.
- **Use cases for sink**:
  - Passing the event-emitting interface to other classes
  - Decoupling the event source from the stream management
  - Creating typed event emitters

### **StreamController in Flutter**

```dart
import 'dart:async';
import 'package:flutter/material.dart';

// Example: Counter with StreamController
class CounterBloc {
  final _countController = StreamController<int>();
  final _eventController = StreamController<CounterEvent>();
  
  // Output stream (read-only)
  Stream<int> get countStream => _countController.stream;
  
  // Input sink (for adding events)
  Sink<CounterEvent> get eventSink => _eventController.sink;
  
  int _count = 0;
  
  CounterBloc() {
    // Listen to events and update state
    _eventController.stream.listen((event) {
      switch (event) {
        case CounterEvent.increment:
          _count++;
          break;
        case CounterEvent.decrement:
          _count--;
          break;
        case CounterEvent.reset:
          _count = 0;
          break;
      }
      // Emit new count
      _countController.add(_count);
    });
  }
  
  void dispose() {
    _countController.close();
    _eventController.close();
  }
}

enum CounterEvent { increment, decrement, reset }

// Counter widget using the bloc
class CounterScreen extends StatefulWidget {
  @override
  _CounterScreenState createState() => _CounterScreenState();
}

class _CounterScreenState extends State<CounterScreen> {
  late final CounterBloc _bloc;
  
  @override
  void initState() {
    super.initState();
    _bloc = CounterBloc();
  }
  
  @override
  void dispose() {
    _bloc.dispose();
    super.dispose();
  }
  
  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(title: Text('StreamController Counter')),
      body: Center(
        child: StreamBuilder<int>(
          stream: _bloc.countStream,
          initialData: 0,
          builder: (context, snapshot) {
            return Column(
              mainAxisAlignment: MainAxisAlignment.center,
              children: [
                Text(
                  'Count',
                  style: TextStyle(fontSize: 24),
                ),
                SizedBox(height: 16),
                Text(
                  '${snapshot.data}',
                  style: TextStyle(
                    fontSize: 72,
                    fontWeight: FontWeight.bold,
                  ),
                ),
                SizedBox(height: 32),
                Row(
                  mainAxisAlignment: MainAxisAlignment.center,
                  children: [
                    FloatingActionButton(
                      heroTag: 'decrement',
                      child: Icon(Icons.remove),
                      onPressed: () {
                        _bloc.eventSink.add(CounterEvent.decrement);
                      },
                    ),
                    SizedBox(width: 16),
                    FloatingActionButton(
                      heroTag: 'reset',
                      child: Icon(Icons.refresh),
                      onPressed: () {
                        _bloc.eventSink.add(CounterEvent.reset);
                      },
                    ),
                    SizedBox(width: 16),
                    FloatingActionButton(
                      heroTag: 'increment',
                      child: Icon(Icons.add),
                      onPressed: () {
                        _bloc.eventSink.add(CounterEvent.increment);
                      },
                    ),
                  ],
                ),
              ],
            );
          },
        ),
      ),
    );
  }
}
```

**Explanation:**

- **BLoC pattern**: This example demonstrates a simplified Business Logic Component (BLoC) pattern using `StreamController`.
- **Separation of concerns**: The `CounterBloc` handles business logic, while the `CounterScreen` handles UI. They communicate via streams.
- **Input/Output streams**:
  - `_eventController`: Receives events (input stream)
  - `_countController`: Emits state changes (output stream)
- **Event-driven**: The widget sends events (increment, decrement, reset) to the bloc via `eventSink`.
- **State propagation**: The bloc processes events and emits new state values via `countStream`.
- **Reactive UI**: `StreamBuilder` automatically updates the UI when the bloc emits new state.
- **Encapsulation**: The internal state (`_count`) is private. External code interacts only via streams (events and state).
- **Lifecycle**: The bloc is created in `initState()` and disposed in `dispose()` to prevent memory leaks.
- **Benefits**:
  - Clear separation of UI and business logic
  - Testable logic (can test bloc without UI)
  - Reactive, responsive UI
  - Predictable state management
- **Real-world application**: This pattern scales well to complex applications with multiple features and state sources.

---

## **6.7 Isolates and Compute-Intensive Operations**

Dart is single-threaded, but you can use Isolates to run code in separate threads. Isolates are essential for CPU-intensive operations that would otherwise block the UI.

### **Understanding Isolates**

```dart
import 'dart:async';
import 'dart:isolate';

void main() async {
  print('=== Single-Threaded Blocking Operation ===');
  await blockingOperation();
  
  print('\n=== Isolated Non-Blocking Operation ===');
  await isolatedOperation();
}

// Blocking operation (BAD - freezes the UI)
Future<void> blockingOperation() async {
  print('Start: Blocking operation');
  
  var start = DateTime.now();
  
  // This blocks the thread
  var result = fibonacci(40);
  
  var duration = DateTime.now().difference(start);
  
  print('Result: $result');
  print('Duration: ${duration.inMilliseconds}ms');
  print('During this time, nothing else could run!');
}

// Non-blocking operation using Isolate (GOOD)
Future<void> isolatedOperation() async {
  print('Start: Isolated operation');
  
  var start = DateTime.now();
  
  // Run fibonacci in a separate isolate
  var result = await Isolate.run(() => fibonacci(40));
  
  var duration = DateTime.now().difference(start);
  
  print('Result: $result');
  print('Duration: ${duration.inMilliseconds}ms');
  print('Main thread was free during computation!');
}

// Compute-intensive function
int fibonacci(int n) {
  if (n <= 1) return n;
  return fibonacci(n - 1) + fibonacci(n - 2);
}
```

**Explanation:**

- **Isolates**: Isolates are independent workers that run in their own memory space. They don't share memory with the main isolate (the main thread).
- **Communication**: Isolates communicate by passing messages, not by sharing memory. This prevents race conditions and makes code safer.
- **`Isolate.run()`**: A convenience method that spawns an isolate, runs a function in it, and returns the result as a Future. The isolate is automatically cleaned up after completion.
- **Blocking vs. Non-blocking**:
  - Blocking operation: The main thread is occupied during computation, freezing the UI.
  - Isolated operation: The computation runs in a separate thread. The main thread remains free to handle UI updates.
- **Memory isolation**: Each isolate has its own memory space. Variables in one isolate are not accessible in another. Data must be passed via messages.
- **Overhead**: Spawning an isolate has overhead (creating memory, setting up communication). It's only worth it for CPU-intensive tasks that take longer than ~10ms.
- **Use cases for Isolates**:
  - Image processing
  - JSON parsing of large payloads
  - Cryptographic operations
  - Complex calculations
  - Data compression/decompression

### **Creating and Communicating with Isolates**

```dart
import 'dart:async';
import 'dart:isolate';

void main() async {
  print('=== Basic Isolate Communication ===');
  await basicCommunication();
  
  print('\n=== Two-Way Isolate Communication ===');
  await twoWayCommunication();
  
  print('\n=== Isolate with Complex Data ===');
  await complexDataIsolate();
}

// Basic isolate communication
Future<void> basicCommunication() async {
  // Create a receive port
  var receivePort = ReceivePort();
  
  // Spawn the isolate
  await Isolate.spawn(
    isolateEntryPoint,
    receivePort.sendPort,
  );
  
  // Wait for the isolate to send a message
  var message = await receivePort.first as String;
  
  print('Received from isolate: $message');
  
  // Close the receive port
  receivePort.close();
}

// Isolate entry point
void isolateEntryPoint(SendPort sendPort) {
  // This runs in the isolate
  var result = heavyComputation();
  
  // Send result back to main isolate
  sendPort.send('Computation complete: $result');
}

// Heavy computation (runs in isolate)
int heavyComputation() {
  var sum = 0;
  for (var i = 0; i < 1000000000; i++) {
    sum += i;
  }
  return sum;
}

// Two-way communication with isolates
Future<void> twoWayCommunication() async {
  var mainReceivePort = ReceivePort();
  
  // Spawn isolate and get its send port
  var isolate = await Isolate.spawn(
    twoWayIsolateEntryPoint,
    mainReceivePort.sendPort,
  );
  
  // Wait for isolate to send its receive port
  var isolateSendPort = await mainReceivePort.first as SendPort;
  
  // Create a response port for this request
  var responsePort = ReceivePort();
  
  // Send a request to the isolate
  isolateSendPort.send({
    'port': responsePort.sendPort,
    'number': 42,
  });
  
  // Wait for the response
  var response = await responsePort.first as Map;
  
  print('Response from isolate:');
  print('  Number: ${response['number']}');
  print('  Doubled: ${response['doubled']}');
  print('  Squared: ${response['squared']}');
  
  // Cleanup
  responsePort.close();
  mainReceivePort.close();
  isolate.kill();
}

// Two-way isolate entry point
void twoWayIsolateEntryPoint(SendPort mainSendPort) {
  // Create a receive port for this isolate
  var isolateReceivePort = ReceivePort();
  
  // Send the receive port to main isolate
  mainSendPort.send(isolateReceivePort.sendPort);
  
  // Listen for requests
  isolateReceivePort.listen((message) {
    if (message is Map) {
      var port = message['port'] as SendPort;
      var number = message['number'] as int;
      
      // Process the request
      port.send({
        'number': number,
        'doubled': number * 2,
        'squared': number * number,
      });
    }
  });
}

// Isolate with complex data
Future<void> complexDataIsolate() async {
  var receivePort = ReceivePort();
  
  await Isolate.spawn(
    complexDataIsolateEntryPoint,
    receivePort.sendPort,
  );
  
  var result = await receivePort.first as Map;
  
  print('Processed data:');
  print('  Count: ${result['count']}');
  print('  Sum: ${result['sum']}');
  print('  Average: ${result['average']}');
  
  receivePort.close();
}

// Complex data isolate entry point
void complexDataIsolateEntryPoint(SendPort sendPort) {
  // Simulate processing a large dataset
  var data = List.generate(1000000, (i) => i);
  
  var count = data.length;
  var sum = data.reduce((a, b) => a + b);
  var average = sum / count;
  
  sendPort.send({
    'count': count,
    'sum': sum,
    'average': average,
  });
}
```

**Explanation:**

- **`ReceivePort`**: A port that can receive messages from other isolates. It's like a mailbox where other isolates can send messages.
- **`SendPort`**: A port that can send messages to a specific `ReceivePort` in another isolate. You get a `SendPort` from a `ReceivePort`.
- **`Isolate.spawn()`**: Spawns a new isolate and runs the given entry point function. The function receives one argument (typically a `SendPort`).
- **Communication flow**:
  1. Main isolate creates a `ReceivePort`
  2. Main isolate spawns the worker isolate, passing the `ReceivePort`'s `SendPort`
  3. Worker isolate sends messages back using the `SendPort`
  4. Main isolate receives messages via the `ReceivePort`
- **Two-way communication**:
  1. Main isolate spawns worker isolate with a `ReceivePort`
  2. Worker isolate creates its own `ReceivePort` and sends its `SendPort` to main isolate
  3. Now both isolates can send messages to each other
  4. Main isolate sends requests including a response `SendPort`
  5. Worker isolate processes and sends response to the provided port
- **Message types**: You can send any serializable object between isolates:
  - Primitives (int, double, String, bool)
  - Lists and Maps
  - Custom objects (must be serializable)
  - `SendPort` objects
- **Cleanup**: Always close `ReceivePort` objects when done to prevent memory leaks. Use `isolate.kill()` to terminate an isolate if needed.

### **Using the `compute` Function**

Flutter provides a `compute` function that simplifies running code in isolates. It's perfect for one-off computations.

```dart
import 'package:flutter/foundation.dart';
import 'dart:async';

void main() async {
  print('=== Using compute function ===');
  await computeExample();
  
  print('\n=== Compute with Custom Object ===');
  await computeWithObject();
}

// Using compute for simple calculation
Future<void> computeExample() async {
  print('Starting computation...');
  
  var start = DateTime.now();
  
  // Run fibonacci in an isolate
  var result = await compute(fibonacci, 40);
  
  var duration = DateTime.now().difference(start);
  
  print('Result: $result');
  print('Duration: ${duration.inMilliseconds}ms');
}

// Fibonacci function (must be top-level or static)
int fibonacci(int n) {
  if (n <= 1) return n;
  return fibonacci(n - 1) + fibonacci(n - 2);
}

// Compute with custom object
Future<void> computeWithObject() async {
  var data = ProcessingData(
    items: List.generate(1000, (i) => i),
    multiplier: 2,
  );
  
  print('Starting data processing...');
  
  var start = DateTime.now();
  
  var result = await compute(processData, data);
  
  var duration = DateTime.now().difference(start);
  
  print('Processed ${result.count} items');
  print('Sum: ${result.sum}');
  print('Duration: ${duration.inMilliseconds}ms');
}

// Processing data (must be top-level or static)
ProcessingResult processData(ProcessingData data) {
  var count = 0;
  var sum = 0;
  
  for (var item in data.items) {
    var processed = item * data.multiplier;
    count++;
    sum += processed;
  }
  
  return ProcessingResult(
    count: count,
    sum: sum,
  );
}

// Data model for processing
class ProcessingData {
  final List<int> items;
  final int multiplier;
  
  ProcessingData({required this.items, required this.multiplier});
}

// Result model
class ProcessingResult {
  final int count;
  final int sum;
  
  ProcessingResult({required this.count, required this.sum});
}
```

**Explanation:**

- **`compute()` function**: A Flutter utility that spawns an isolate, runs a function, and returns the result. It's simpler than manually managing isolates.
- **Function requirements**: The function passed to `compute` must be:
  - A top-level function (not a method or closure)
  - Or a static method
  - This is because the function needs to be serializable to send to the isolate
- **Parameters**: `compute` takes two arguments:
  1. The function to run
  2. The argument to pass to that function
- **Return value**: The function's return value is sent back to the main isolate as a Future.
- **Automatic cleanup**: `compute` automatically handles isolate creation, message passing, and cleanup. You don't need to manage ports or kill isolates.
- **Serializability**: All parameters and return values must be serializable. Custom classes work as long as they don't contain non-serializable types (like open files, sockets, or other isolates).
- **Single use**: Each `compute` call spawns a new isolate. For repeated computations, consider using a long-running isolate with custom message passing.
- **Performance**: `compute` has overhead (creating isolate, serializing data). Use it for tasks that take longer than ~10-20ms to be worth it.
- **Best practice**: Use `compute` for:
  - JSON decoding of large responses
  - Image processing
  - Data filtering/sorting of large lists
  - Any computation that takes more than a frame (16ms)

### **Real-World Example: Image Processing in Isolate**

```dart
import 'dart:async';
import 'dart:isolate';
import 'dart:ui' as ui;
import 'package:flutter/material.dart';
import 'package:flutter/services.dart';

// Image processing using isolates
class ImageProcessingScreen extends StatefulWidget {
  @override
  _ImageProcessingScreenState createState() => _ImageProcessingScreenState();
}

class _ImageProcessingScreenState extends State<ImageProcessingScreen> {
  ui.Image? _originalImage;
  ui.Image? _processedImage;
  bool _isProcessing = false;
  String _status = 'Ready';

  @override
  void initState() {
    super.initState();
    _loadImage();
  }

  Future<void> _loadImage() async {
    try {
      // Load image from assets
      final byteData = await rootBundle.load('assets/sample_image.jpg');
      final codec = await ui.instantiateImageCodec(
        byteData.buffer.asUint8List(),
      );
      final frame = await codec.getNextFrame();
      
      setState(() {
        _originalImage = frame.image;
      });
    } catch (e) {
      setState(() {
        _status = 'Error loading image: $e';
      });
    }
  }

  Future<void> _processImageInIsolate() async {
    if (_originalImage == null) return;
    
    setState(() {
      _isProcessing = true;
      _status = 'Processing...';
    });
    
    try {
      // Convert image to byte data
      final byteData = await _originalImage!.toByteData(
        format: ui.ImageByteFormat.png,
      );
      
      if (byteData == null) {
        throw Exception('Failed to get image byte data');
      }
      
      // Process in isolate
      final processedBytes = await Isolate.run(
        () => _applyGrayscaleFilter(byteData.buffer.asUint8List()),
      );
      
      // Create image from processed bytes
      final codec = await ui.instantiateImageCodec(processedBytes);
      final frame = await codec.getNextFrame();
      
      setState(() {
        _processedImage = frame.image;
        _status = 'Processing complete!';
      });
    } catch (e) {
      setState(() {
        _status = 'Error processing image: $e';
      });
    } finally {
      setState(() {
        _isProcessing = false;
      });
    }
  }

  Future<void> _processImageWithCompute() async {
    if (_originalImage == null) return;
    
    setState(() {
      _isProcessing = true;
      _status = 'Processing with compute...';
    });
    
    try {
      // Convert image to byte data
      final byteData = await _originalImage!.toByteData(
        format: ui.ImageByteFormat.png,
      );
      
      if (byteData == null) {
        throw Exception('Failed to get image byte data');
      }
      
      // Process using compute
      final processedBytes = await compute(
        _applyGrayscaleFilter,
        byteData.buffer.asUint8List(),
      );
      
      // Create image from processed bytes
      final codec = await ui.instantiateImageCodec(processedBytes);
      final frame = await codec.getNextFrame();
      
      setState(() {
        _processedImage = frame.image;
        _status = 'Processing complete!';
      });
    } catch (e) {
      setState(() {
        _status = 'Error processing image: $e';
      });
    } finally {
      setState(() {
        _isProcessing = false;
      });
    }
  }

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(title: Text('Image Processing with Isolates')),
      body: SingleChildScrollView(
        padding: EdgeInsets.all(16),
        child: Column(
          crossAxisAlignment: CrossAxisAlignment.stretch,
          children: [
            // Status
            Card(
              child: Padding(
                padding: EdgeInsets.all(16),
                child: Row(
                  children: [
                    if (_isProcessing)
                      Padding(
                        padding: EdgeInsets.only(right: 16),
                        child: SizedBox(
                          width: 20,
                          height: 20,
                          child: CircularProgressIndicator(strokeWidth: 2),
                        ),
                      ),
                    Expanded(
                      child: Text(
                        _status,
                        style: TextStyle(fontSize: 16),
                      ),
                    ),
                  ],
                ),
              ),
            ),
            
            SizedBox(height: 16),
            
            // Original image
            if (_originalImage != null) ...[
              Text(
                'Original Image',
                style: TextStyle(fontSize: 18, fontWeight: FontWeight.bold),
              ),
              SizedBox(height: 8),
              Card(
                child: Padding(
                  padding: EdgeInsets.all(8),
                  child: RawImage(
                    image: _originalImage,
                    width: 300,
                    fit: BoxFit.contain,
                  ),
                ),
              ),
              SizedBox(height: 16),
            ],
            
            // Processed image
            if (_processedImage != null) ...[
              Text(
                'Processed Image (Grayscale)',
                style: TextStyle(fontSize: 18, fontWeight: FontWeight.bold),
              ),
              SizedBox(height: 8),
              Card(
                child: Padding(
                  padding: EdgeInsets.all(8),
                  child: RawImage(
                    image: _processedImage,
                    width: 300,
                    fit: BoxFit.contain,
                  ),
                ),
              ),
              SizedBox(height: 16),
            ],
            
            // Buttons
            Row(
              children: [
                Expanded(
                  child: ElevatedButton(
                    onPressed: _isProcessing || _originalImage == null
                        ? null
                        : _processImageInIsolate,
                    child: Text('Process with Isolate.run'),
                  ),
                ),
                SizedBox(width: 8),
                Expanded(
                  child: ElevatedButton(
                    onPressed: _isProcessing || _originalImage == null
                        ? null
                        : _processImageWithCompute,
                    child: Text('Process with compute'),
                  ),
                ),
              ],
            ),
          ],
        ),
      ),
    );
  }
}

// Grayscale filter function (must be top-level or static)
Uint8List _applyGrayscaleFilter(Uint8List bytes) {
  // This is a simplified grayscale filter
  // In a real app, you'd parse the image format and process pixels
  
  var result = Uint8List(bytes.length);
  
  for (var i = 0; i < bytes.length; i += 4) {
    if (i + 3 < bytes.length) {
      // RGBA values
      var r = bytes[i];
      var g = bytes[i + 1];
      var b = bytes[i + 2];
      var a = bytes[i + 3];
      
      // Calculate grayscale (luminosity method)
      var gray = (0.299 * r + 0.587 * g + 0.114 * b).round();
      
      result[i] = gray;
      result[i + 1] = gray;
      result[i + 2] = gray;
      result[i + 3] = a; // Keep alpha channel
    }
  }
  
  return result;
}
```

**Explanation:**

- **Image processing workflow**:
  1. Load image from assets
  2. Convert to byte data
  3. Process byte data in isolate (grayscale filter)
  4. Convert processed bytes back to image
  5. Display result
- **`toByteData()`**: Converts a `ui.Image` to byte data that can be sent to an isolate. Images can't be sent directly because they contain native resources.
- **`instantiateImageCodec()`**: Decodes image bytes back into a `ui.Image`. This runs on the main thread because it interacts with the Flutter engine.
- **Grayscale algorithm**: A simple luminosity-based grayscale filter. Uses the formula: gray = 0.299×R + 0.587×G + 0.114×B.
- **Byte manipulation**: Processes the raw byte array representing the image. Each pixel is typically 4 bytes (RGBA).
- **UI responsiveness**: Processing happens in an isolate, so the UI remains responsive during the operation.
- **Loading indicator**: Shows a spinner while processing to provide feedback to the user.
- **Two approaches**: Shows both `Isolate.run()` and `compute()` for comparison.
- **Error handling**: Catches and displays errors at each step.
- **Real-world considerations**:
  - This is a simplified filter. Real image processing would use proper image libraries.
  - Large images might exceed isolate message size limits.
  - Consider streaming/chunking for very large data.
- **Best practices**:
  - Keep isolate functions pure (no side effects)
  - Handle memory carefully with large images
  - Provide user feedback during long operations
  - Clean up resources properly

---

## **Chapter Summary**

In this chapter, we covered Dart's asynchronous programming model, essential for building responsive Flutter applications:

### **Key Takeaways:**

1. **Event Loop**: Dart uses a single-threaded event loop. Understanding execution order (sync → microtasks → events) is crucial for writing efficient async code.
2. **Futures**: Represent a single value that arrives asynchronously. Use `Future.value()`, `Future.error()`, `Future.delayed()`, and `Future.wait()`.
3. **Async/Await**: Provides synchronous-looking syntax for asynchronous code. Always use for readability over `.then()` callbacks.
4. **Error Handling**: Use try-catch-finally with async/await. Handle errors properly to prevent crashes.
5. **Streams**: Represent a sequence of asynchronous events over time. Essential for real-time data and continuous operations.
6. **Stream Creation**: Use `Stream.fromIterable()`, `Stream.fromFuture()`, `Stream.periodic()`, `StreamController`, and `Stream.value()`.
7. **Stream Transformations**: Use `map`, `where`, `take`, `skip`, `expand`, `asyncMap`, and `asyncExpand` to transform streams.
8. **StreamBuilder**: Flutter widget for building reactive UIs based on stream data. Handles loading, error, and success states automatically.
9. **StreamController**: Allows creating custom streams with full control over when values, errors, and done events are emitted.
10. **Isolates**: Separate execution threads for CPU-intensive operations. Use `Isolate.run()` or `compute()` to avoid blocking the UI.
11. **Performance**: Always use asynchronous operations for network requests, file I/O, and heavy computations to keep the UI responsive.

### **Best Practices:**

- Use async/await for readable, maintainable code
- Handle all error cases in asynchronous operations
- Use `Future.wait()` for parallel independent operations
- Prefer streams over callbacks for continuous data flows
- Use `StreamBuilder` for reactive UI updates in Flutter
- Use isolates for CPU-intensive tasks (>10-20ms)
- Always clean up subscriptions and controllers to prevent memory leaks
- Provide user feedback (loading indicators, error messages) during async operations

### **Next Steps:**

Now that you understand asynchronous programming in Dart, the next chapter will cover **Part III: Flutter Widget Deep Dive**, including:

- Widget fundamentals and the widget tree
- Layout and composition (Container, Row, Column, Stack)
- Material Design and Cupertino widgets
- Input, forms, and validation

---

**End of Chapter 6**

---

# **Next Chapter: Chapter 7 - Widget Fundamentals**

Chapter 7 will explore Flutter's widget-based architecture, understanding how widgets work, the widget tree, and the fundamental concepts of building Flutter UIs.